In [33]:
import spotify_auth
import pprint
import youtube_auth

In [ ]:
def get_playlist_tracks(playlist_id): # Put the names of the songs in our playlist into an array 
    names = []

    results = spotify_auth.sp.playlist(playlist_id)
    
    tracks = results['tracks']['items']
    
    for item in tracks:
        names.append(item['track']['name'])
    
    return names

In [28]:
# Crack playlist ID: 3xWx987zAhFtL1r5kPhtpr

crack = get_playlist_tracks("3xWx987zAhFtL1r5kPhtpr")
print(crack)

['Rodeo (Remix)', 'One Of The Girls (with JENNIE, Lily Rose Depp)', 'Confident', 'Her Way', 'Belong to the City', 'PERSIAN RUGS', 'FREAK IN YOU', 'Earned It (Fifty Shades Of Grey)', 'Streets', 'Wine Pon You (feat. Konshens)', 'Girls Need Love (with Drake) - Remix', "Don't", 'Wus Good / Curious', 'Coming Down', 'The Party & The After Party', 'What You Need', 'Sativa', 'Feel It', 'Sex With Me', 'Needed Me', 'Hotel', 'Woo', 'Lost in the Fire (feat. The Weeknd)', 'Often', 'Into It', "Sun Down, I'm Up - Sped Up & Pitched", 'Call Out My Name', 'Cherry Hill', 'All The Time', 'Excitement', 'One Of The Girls (with JENNIE, Lily Rose Depp)', 'Wine Pon You (feat. Konshens)', 'Break from Toronto', "Don't Run", 'Dreamin', 'SOMETHING ABOUT YOU', 'TBH', 'GREEDY', 'Come and See Me (feat. Drake)', 'Recognize (feat. Drake)', 'THINGS & SUCH', 'Say It', "Let Em' Know", 'Poison', 'Been Away', 'Forever Yours', 'Chanel', 'R e s e n t m e n t', 'N o C h i l l', 'Stay Ready (What A Life)', 'Grown Woman', 'WEST 

In [31]:
def search_song(query):
    result = youtube_auth.youtube.search().list(part="snippet", q=query, type="video", maxResults=1).execute()
    # result['items'][0]['snippet']['title'] --> title of song
    return result

pprint.pprint(search_song("Some of Your Love PND Slowed Reverb"))

{'etag': 'vCMFW5JuhYUZBHb_MRQaF1IPIeA',
 'items': [{'etag': 'RfHpTuCkqOe3jtWbbfHGC-3-bW8',
            'id': {'kind': 'youtube#video', 'videoId': 'lovoL6CF604'},
            'kind': 'youtube#searchResult',
            'snippet': {'channelId': 'UC3UAU2yjg2_45PgGZHWJu3Q',
                        'channelTitle': 'LxLiberty',
                        'description': 'Enjoy ! original song : '
                                       'https://www.youtube.com/watch?v=0vWkvXv5s_g '
                                       'I do not own the rights to the '
                                       'original song. Leave your ...',
                        'liveBroadcastContent': 'none',
                        'publishTime': '2019-03-11T22:05:38Z',
                        'publishedAt': '2019-03-11T22:05:38Z',
                        'thumbnails': {'default': {'height': 90,
                                                   'url': 'https://i.ytimg.com/vi/lovoL6CF604/default.jpg',
                        

In [ ]:
# This will return an array of youtube-video ids of slowed-reverb version of songs 
def ids_slowed_version(spotify_playlist):
    slowed_version = []
    for song in spotify_playlist[:3]:
        slowed_version.append(search_song(song + " Slowed Reverb"))

    ids = [video['items'][0]['id']['videoId'] for video in slowed_version]
    
    return ids

pprint.pprint(ids_slowed_version(crack))

['6rp-a910-XI', 'I7IYfWhHGZw', 'HW_d3_kyReo']


In [ ]:
# Creates a playlist for you

def create_playlist(title, description=None, privacyStatus='private', defaultLanguage=None):
    snippet_body = {
        'snippet': {
            'title': title,
            'description': description,
            'defaultLanguage': defaultLanguage
        },
        'status': {
            'privacyStatus': privacyStatus
        }
    }
    return youtube_auth.youtube.playlists().insert(part="snippet,status", body=snippet_body).execute()